In [3]:
import pandas as pd
import requests
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import fasttext

In [ ]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.bin.gz"

In [3]:
!gunzip --decompress cc.ru.300.bin.gz

In [5]:
model = fasttext.load_model('cc.ru.300.bin')

In [ ]:
!wget https://raw.githubusercontent.com/dhhse/fanfics4hack/main/data/FanFics_Metadata_full.json

In [108]:
data = pd.read_json('FanFics_Metadata_full.json')

features_columns = [
       'Genre', 'Rating', 'Status', 
       'Title', 'Thanks', 'Author note'
]

data = data.dropna(subset=features_columns)
data = data.drop(data.index[200:])
data = data.reset_index()

In [180]:
class DFWrapper:
    def __init__(self, dataframe, t_cl,
                 vectorizer=CountVectorizer(), fasttext_flag=False):
        # self.dataframe = dataframe.dropna() это нужно оставить в
        # черновом варианте, но тестовый дф тогда будет пустым
        self.dataframe = dataframe
        self.dataframe = self.dataframe.reset_index()
        self.t_cl = self.dataframe[[t_cl]]
        self.f_cls = dataframe.drop(self.t_cl, axis=1)
        self.vectorizer = vectorizer
        self.fasttext_flag = fasttext_flag

    @property
    def features(self):
        nfeatures = self.t_cl  # я честно не разобрался, как сделать
# нормальный df ,у меня какая-то замара с несовместимостью df и series
# результаты фичерс надо куда-то прикрепить
# загружаем модель, если юзер сказал юзать fasttext
        # if self.fasttext_flag:
        #    model = fasttext.load_model('cc.ru.300.bin')

        for column in self.f_cls:
            if isinstance(self.dataframe[column][0], str):
                # я проверяю по первой строке, лежат ли в столбце строки
                if len(self.dataframe[column])*0.1 > \
                       self.dataframe[column].nunique():
                    # я понял это так, что если в столбце значений менее
                    # 10% от всей длины столбца, то тогда dummies
                    dummies = pd.get_dummies(self.dataframe[column])
                    nfeatures = nfeatures.join(dummies)

                else:
                    if self.fasttext_flag:  # если юзер хочет фасттекст
                        vectorized = []
                        for cell in self.dataframe[column]:
                            fasttext_vector = model.get_sentence_vector(
                                cell.replace('\n', ''))
                            vectorized.append(list(fasttext_vector))

                    else:
                        verorises = self.vectorizer.fit_transform(
                            self.dataframe[column])
                        vectorized = verorises.toarray()

                # трансформируем данные
                t = list(zip(*vectorized))
                v_col = {}
                n = 0
                for col in t:
                    v_col['vec_{}_{}'.format(column, n)] = col
                    n += 1

                nfeatures = nfeatures.join(pd.DataFrame(v_col))

        return nfeatures.drop(self.t_cl, axis=1)

    def fit(self, clas):
        f_features = self.features  # не хочу изменять аттрибут
        f_dataframe = f_features.join(self.t_cl)
        data_train, data_test = train_test_split(f_dataframe)
        if isinstance(clas, type(KNeighborsClassifier())) or \
           isinstance(clas, type(LogisticRegression())):
            neight = clas.fit(X=data_train[list(self.features.columns)],
                              y=data_train[list(self.t_cl.columns)[0]])

        return model


df = DFWrapper(data, 'Genre',  vectorizer=False, fasttext_flag=True)

In [181]:
z = df.features

In [182]:
df.fit(KNeighborsClassifier())